In [ ]:
import sys
import os

# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))  # Adjust the path accordingly

In [ ]:
import pandas as pd
import numpy as np
import pickle
from load_data import *
from preprocessing import *

In [ ]:
data_file = "../data/brown-universal.txt"
tags_file = "../data/tags-universal.txt"
predictions_file = "../results/pred-viterbi-tags.txt"

# Load Data from File

In [ ]:
train, test = load_brown_data(data_file, split=0.8)
tags = load_tags(tags_file)

In [ ]:
print("There are {} sentences in the training set.".format(len(train)))
print("There are {} sentences in the testing set.".format(len(test)))

In [ ]:
# partition train so only a few of the samples are used for the initial probabilities
train_sample = train

# Train HMM with NLTK

In [ ]:
! pip install -U nltk

In [ ]:
import nltk

def laplace_smoothing(freq_dist, bins):
    return nltk.LaplaceProbDist(freq_dist, bins)

def trainer(data):
    trainer = nltk.tag.hmm.HiddenMarkovModelTrainer()
    hmm_tagger = trainer.train_supervised(data, estimator=laplace_smoothing)
    return hmm_tagger

data = [[(token.get_word(), token.get_pos()) for token in sentence] for sentence in train_sample]
hmm_tagger = trainer(data)

In [ ]:
save the hmm so we don't have to train every time

# Save the trained HMM model to a file
with open("../results/hmm_tagger-NLTK.pkl", "wb") as f:
    pickle.dump(hmm_tagger, f)
    f.close()